# Horizon examples

Welcome to this live introduction to [**Horizon**](https://github.com/ADVRHumanoids/horizon)!

Horizon is a framework for trajectory optimization and optimal control tailored to robotic systems.
It is particulary useful to describe and solve parametric nolinear problems for trajectory planning, both off-line and in a MPC fashion.

Horizon is based on [**CasADi**](https://web.casadi.org/), a powerful tool for nonlinear optimization and algorithmic differentiation.

You can explore the [**documentation**](https://advrhumanoids.github.io/horizon) to learn more about this framework.

## Getting started

This session allows to run some examples demonstrating the capabilities of the framework.
If you are not familiar with the JupyterLab environment, follow this brief [guide](jupi_guide.ipynb).

In a new terminal, just enter the following commands:

 - **python3 -m horizon.examples.cart_pole --r**

where --r activate the replay mode, which allows to visualize the optimized trajectory on the robot (for a complete list of options, type *--help*)  

Once the optimal solution is found, the example will automatically replay it on the robot.
To visualize it, it is enough to open the zethus visualizer (rviz), and select the robot description.

There are many examples that can be run, involving different robots (spot, quadruped template, roped robot, cart pole...) performing different actions.

A few possible commands:

 - **python3 -m horizon.examples.cart_pole --r --minimize_t**
 - **python3 -m horizon.examples.spot --r --action leap**
 - **python3 -m horizon.examples.roped_robot --r --action rappel**